In [1]:
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt

from astropy.modeling import fitting
from sunkit_spex.models.physical.albedo import Albedo
from sunkit_spex.models.physical.nonthermal import ThickTarget
from plotting.plotter import plot_fit_results
from sunkit_spex.models.physical.thermal import ThermalEmission
from sunkit_spex.models.scaling import InverseSquareFluxScaling
from sunkit_spex.models.instrument_response import MatrixModel
from sunkit_spex.spectrum.spectrum import SpectralAxis
from sunkit_spex.spectrum.spectrum import Spectrum
from sunkit_spex.extern.stix import STIXLoader


In [ ]:
dir = 'data/210507/'
spec = STIXLoader(spectrum_file=dir+'stx_spectrum_2105070034.fits',srm_file=dir+'stx_srm_2105070034.fits')

spec

In [ ]:
start = "2021-05-07T18:53:00"
end = "2021-05-07T18:53:20"

spec.start_event_time=start
spec.end_event_time=end

fig = plt.figure(figsize=(9,6))
spec.lightcurve(energy_ranges=[[4,10], [10,30], [25,50]])
plt.show()

In [5]:

count_channel_bins = spec['count_channel_bins']
indices_fit = np.where( (count_channel_bins[:,1] >= 5.0)  & (count_channel_bins[:,1] <= 55.0) )[0]
counts = (spec['count_rate'][indices_fit] *u.ct *u.keV**-1 *u.s**-1) 
counts_err = np.array(spec['count_rate_error'][indices_fit]) 

srm = spec['srm'][:,indices_fit]


In [ ]:
obj_spec_photon  = Spectrum(counts,uncertainty=counts_err,spectral_axis=spec['photon_channel_bins']*u.keV)
obj_spec_counts = Spectrum(counts,uncertainty=counts_err,spectral_axis=spec['count_channel_bins'][indices_fit,:]*u.keV)

ph_energies_centers =  SpectralAxis._centers_from_edges(obj_spec_photon._spectral_axis._bin_edges)
counts_energies_centers =  SpectralAxis._centers_from_edges(obj_spec_counts._spectral_axis._bin_edges)


In [7]:
srm_model = MatrixModel(
    matrix=srm, input_axis=obj_spec_photon._spectral_axis._bin_edges, output_axis=obj_spec_counts._spectral_axis._bin_edges, c=1 * u.ct *u.cm**2 * u.ph**-1, 
    _input_units={"x": u.ph *u.keV**-1 *u.s**-1 * u.cm**-2}, _output_units={"y": u.ct* u.keV**-1 * u.s**-1}
)

In [16]:
distance = InverseSquareFluxScaling(1*u.AU)
f_vth = ThermalEmission()
thick = ThickTarget(break_energy=1500*u.keV,low_e_cutoff=20*u.keV)
albedo = Albedo(energy_edges=obj_spec_photon._spectral_axis._bin_edges, theta=45*u.deg)

ph_model_4fit = (((f_vth + thick) * distance ) | albedo)  | srm_model


In [17]:
ph_model_4fit.theta_3.fixed = True

ph_model_4fit.temperature_0.fixed = False
ph_model_4fit.emission_measure_0.fixed = False

ph_model_4fit.break_energy_1.fixed= True
ph_model_4fit.low_e_cutoff_1.fixed = False
ph_model_4fit.q_1.fixed = False
ph_model_4fit.total_eflux_1.fixed = False


In [18]:
pfit = fitting.TRFLSQFitter()


new_model = pfit(ph_model_4fit,obj_spec_photon._spectral_axis._bin_edges, obj_spec_photon.data,
                 weights=1/obj_spec_photon.uncertainty.array,maxiter=100000)

In [ ]:
print(new_model)

In [ ]:
save_name = 'Sunkit-Spex_output.pdf'
fit_times = f'{start} - {end}'

plot_fit_results(obj_spec_counts._spectral_axis._bin_edges,obj_spec_photon._spectral_axis._bin_edges,
                 obj_spec_counts.data << obj_spec_counts.unit,obj_spec_counts.uncertainty.array<< obj_spec_counts.unit,
                 new_model,save_name,fit_times)

In [ ]:
new_model['ThermalEmission'](obj_spec_photon._spectral_axis._bin_edges) 

In [ ]:
ph_model_4fit['ThermalEmission'](obj_spec_photon._spectral_axis._bin_edges) 

In [ ]:
new_model.fixed